In [1]:
from pii_masker import run_test
from pii_masker.tests_runner import load_tests_bundle, run_tests_from_json

In [16]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Default device for new CPU tensor:", torch.tensor([1]).device)

# Example: model device
# print("Model device:", next(model.parameters()).device)

if torch.cuda.is_available():
    x = torch.randn(2, 3, device="cuda")
    print("x device:", x.device)
    print("GPU name:", torch.cuda.get_device_name(0))

CUDA available: False
Default device for new CPU tensor: cpu


In [23]:
model_instruction = """
Note: The following message contains intentionally masked information using placeholders 
(e.g. [PERSON_1], [ACCOUNT_ID_1], [TRANSACTION_ID_1]).

Please:
- Treat each placeholder as a consistent anonymous entity.
- Refer back to the same placeholder when discussing actions or events related to it.
- Do not invent names, identities, or additional personal details.
- Use the masked information as provided to respond to the user’s query."""

In [17]:
text = """
Subject: URGENT: Unauthorised transaction dispute - Ref #99281 From: Sarah Jenkins sarah.jenkins.1982@example.co.ukTo: fraud-support@fintech-bank.co.uk

Hi Support Team,

I am writing to urgently dispute a transaction I did not make. I believe my card details were compromised at a shop in London yesterday.

Here are my details for verification:Full Name: Sarah Louise Jenkins DOB: 14/03/1982 Current Address: Flat 4, 152 Oxford Street, London, W1D 1BS National Insurance No: QQ 12 34 56 A

The transaction in question is for £450.00 charged to my Premium Debit Card.Card Number: 4751 2201 5521 4422 Expiry: 09/27 CVV: 342

Please refund this immediately to my main current account:Sort Code: 04-00-04 Account Number: 12345678

You can reach me on my mobile at 07700 900123 if you need more info.
"""

masked_text, mapping, scores, spans = run_test(text, threshold=0.7)
print(masked_text)
print()
print("Mapping:")
for k, v in zip(mapping.items(), scores.items()):
    print(f"{k[0]}, {k[1]}: {v[1]}")


Subject: URGENT: Unauthorised transaction dispute - Ref #99281 From: Sarah Jenkins [EMAIL_ADDRESS_1]: [EMAIL_ADDRESS_2]

Hi [ORG_1],

I am writing to urgently dispute a transaction I did not make. I believe my card details were compromised at a shop in London yesterday.

Here are my details for verification:Full Name: [PERSON_1] DOB: [DATE_OF_BIRTH_1] Current Address: [UK_ADDRESS_1] National Insurance No: QQ 12 34 56 A

The transaction in question is for £450.00 charged to my Premium Debit Card.Card Number: [CREDIT_CARD_NUMBER_1] Expiry: [CARD_EXPIRY_1] CVV: 342

Please refund this immediately to my main current account:Sort Code: [UK_SORT_CODE_1] Account Number: [UK_ACCOUNT_NUMBER_1]

You can reach me on my mobile at [UK_PHONE_NUMBER_1] if you need more info.


Mapping:
[EMAIL_ADDRESS_1], sarah.jenkins.1982@example.co.ukTo: 0.99
[EMAIL_ADDRESS_2], fraud-support@fintech-bank.co.uk: 0.99
[ORG_1], Support Team: 0.7363156676292419
[PERSON_1], Sarah Louise Jenkins: 0.9478381276130676
[DAT

In [40]:
text = """You are an assistant helping me investigate a customer complaint and suggest next steps.

Customer details:
Name: Harriet Jane Evans
Date of birth: 14 March 1989
Email: harriet.evans89@gmail.com
Mobile: +44 7911 456 882

Account information:
Bank: Barclays
Sort code: 20-45-67
Account number: 83920145
IBAN: GB29BARC20456783920145
Customer ID: CUST-UK-492817
Transaction ID in question: TXN-77834192

Address on file:
221 King Street, London, SW1A 1AA, United Kingdom

Issue:
Customer claims £2,450 was transferred on 11/01/2026 to an unknown recipient.
Transaction originated from IP address 82.14.233.91.
Login was recorded at 02:13 AM.


"""

masked_text, mapping, scores, spans = run_test(text, threshold=0.7)
print(masked_text)
print()
print("Mapping:")
for k, v in zip(mapping.items(), scores.items()):
    print(f"{k[0]}, {k[1]}: {v[1]}")

You are an assistant helping me investigate a customer complaint and suggest next steps.

Customer details:
Name: [PERSON_1]
Date of birth: [DATE_OF_BIRTH_1]
Email: [EMAIL_ADDRESS_1]
Mobile: [UK_PHONE_NUMBER_1]

Account information:
Bank: [ORG_1]
Sort code: [UK_SORT_CODE_1]
Account number: [UK_ACCOUNT_NUMBER_1]
IBAN: [UK_IBAN_1]
Customer ID: [ACCOUNT_ID_1]
Transaction ID in question: [TRANSACTION_ID_1]

Address on file:
[UK_ADDRESS_1]

Issue:
Customer claims £2,450 was transferred on [DATE_1] to an unknown recipient.
Transaction originated from IP address [IP_ADDRESS_1].
Login was recorded at 02:13 AM.




Mapping:
[PERSON_1], Harriet Jane Evans: 0.9905639886856079
[DATE_OF_BIRTH_1], 14 March 1989: 0.97
[EMAIL_ADDRESS_1], harriet.evans89@gmail.com: 0.99
[UK_PHONE_NUMBER_1], +44 7911 456 882: 0.9988365769386292
[ORG_1], Barclays: 0.8443201184272766
[UK_SORT_CODE_1], 20-45-67: 1.0
[UK_ACCOUNT_NUMBER_1], 83920145: 1.0
[UK_IBAN_1], GB29BARC20456783920145: 0.7776895642280579
[ACCOUNT_ID_1],

In [21]:
text = """You are helping review a customer-reported issue related to a recent account transaction.

Case details:
- A customer (Harriet Jane Evans) reports an outbound transaction they do not recognise.
- The transaction is associated with customer account ID CUST-UK-492817 and reference TXN-77834192.
- The transfer amount is £2,450 and was completed successfully.
- The login session linked to the transaction originated from a new IP address (82.14.233.91).
- The activity occurred outside the customer’s usual usage hours.
- There is no information provided about prior similar incidents.

Tasks:
1. Based only on the patterns described, list two indicators that could suggest account takeover and one indicator that could suggest customer error. Refer to the customer where relevant.
2. Draft a short, neutral response message to the customer acknowledging the report and explaining next steps, without adding any personal details.
3. State whether a temporary account restriction on the account would be a reasonable precaution, and briefly explain why.
4. Identify one internal team (e.g. fraud, risk, operations) that would typically be involved next, and explain their role at a high level."""

masked_text, mapping, scores, spans = run_test(text, threshold=0.7)
print(masked_text)
print()
print("Mapping:")
for k, v in zip(mapping.items(), scores.items()):
    print(f"{k[0]}, {k[1]}: {v[1]}")

You are helping review a customer-reported issue related to a recent account transaction.

Case details:
- A customer ([PERSON_1]) reports an outbound transaction they do not recognise.
- The transaction is associated with customer account ID [TRANSACTION_ID_1] and reference [CUSTOMER_REFERENCE_1].
- The transfer amount is £2,450 and was completed successfully.
- The login session linked to the transaction originated from a new IP address ([IP_ADDRESS_1]).
- The activity occurred outside the customer’s usual usage hours.
- There is no information provided about prior similar incidents.

Tasks:
1. Based only on the patterns described, list two indicators that could suggest account takeover and one indicator that could suggest customer error. Refer to the customer where relevant.
2. Draft a short, neutral response message to the customer acknowledging the report and explaining next steps, without adding any personal details.
3. State whether a temporary account restriction on the account

In [ ]:
https://chatgpt.com/share/696d345c-7f34-8006-b03a-b627b3656a77

https://chatgpt.com/share/696d347f-0b48-8006-a148-cca46e4835df

In [26]:
model_prompt = model_instruction + "\n" + masked_text
print(model_prompt)


Note: The following message contains intentionally masked information using placeholders 
(e.g. [PERSON_1], [ACCOUNT_ID_1], [TRANSACTION_ID_1]).

Please:
- Treat each placeholder as a consistent anonymous entity.
- Refer back to the same placeholder when discussing actions or events related to it.
- Do not invent names, identities, or additional personal details.
- Use the masked information as provided to respond to the user’s query.
You are helping review a customer-reported issue related to a recent account transaction.

Case details:
- A customer ([PERSON_1]) reports an outbound transaction they do not recognise.
- The transaction is associated with customer account ID [TRANSACTION_ID_1] and reference [CUSTOMER_REFERENCE_1].
- The transfer amount is £2,450 and was completed successfully.
- The login session linked to the transaction originated from a new IP address ([IP_ADDRESS_1]).
- The activity occurred outside the customer’s usual usage hours.
- There is no information provide

In [41]:
summary_txt = """Please summarise the case I’ve been working on below.

Case details:

A customer named Sarah Thompson reported an outbound bank transfer of £2,450 that she does not recognise. The transaction reference is FT-98437261 and it completed successfully on 14 January 2026.

The transaction was associated with a login from a new IP address (185.73.44.219) at approximately 02:17 AM, which is outside the customer’s normal usage hours. Sarah Thompson confirmed that she does not recognise the device or login activity.

She still has access to her account and has not identified any other suspicious transactions. This is the first incident of this kind reported by the customer.

Please provide:

A short factual summary of the case

Key risk indicators observed

Recommended immediate next steps"""

masked_text, mapping, scores, spans = run_test(summary_txt, threshold=0.7)
print(masked_text)

Please summarise the case I’ve been working on below.

Case details:

A customer named [PERSON_1] reported an outbound bank transfer of £2,450 that she does not recognise. The transaction reference is [TRANSACTION_ID_1] and it completed successfully on [DATE_1].

The transaction was associated with a login from a new IP address ([IP_ADDRESS_1]) at approximately 02:17 AM, which is outside the customer’s normal usage hours. [PERSON_1] confirmed that she does not recognise the device or login activity.

She still has access to her account and has not identified any other suspicious transactions. This is the first incident of this kind reported by the customer.

Please provide:

A short factual summary of the case

Key risk indicators observed

Recommended immediate next steps


In [42]:
model_prompt = model_instruction + "\n" + masked_text
print(model_prompt)


Note: The following message contains intentionally masked information using placeholders 
(e.g. [PERSON_1], [ACCOUNT_ID_1], [TRANSACTION_ID_1]).

Please:
- Treat each placeholder as a consistent anonymous entity.
- Refer back to the same placeholder when discussing actions or events related to it.
- Do not invent names, identities, or additional personal details.
- Use the masked information as provided to respond to the user’s query.
Please summarise the case I’ve been working on below.

Case details:

A customer named [PERSON_1] reported an outbound bank transfer of £2,450 that she does not recognise. The transaction reference is [TRANSACTION_ID_1] and it completed successfully on [DATE_1].

The transaction was associated with a login from a new IP address ([IP_ADDRESS_1]) at approximately 02:17 AM, which is outside the customer’s normal usage hours. [PERSON_1] confirmed that she does not recognise the device or login activity.

She still has access to her account and has not identi

In [ ]:
meeting_report_txt = """
Please draft a concise internal summary for a board-level report based on the information below.

Case details:

On 14 January 2026, a customer reported an unrecognised outbound transfer of £2,450 (transaction reference FT-98437261). The transaction completed successfully and was linked to a login from IP address 185.73.44.219 at 02:17 AM, outside the customer’s normal activity window.

The customer retains access to the account and no additional suspicious transactions have been identified. The case is currently under review by the fraud team, and temporary preventative controls have been applied.

Please provide:

A short executive summary (3–5 sentences)

Any immediate financial or operational considerations

Current status and next steps"""

masked_text, mapping, scores, spans = run_test(meeting_report_txt, threshold=0.7)
print(masked_text)

Please draft a concise internal summary for a board-level report based on the information below.

Case details:

On [DATE_1], a [PERSON_1] reported an unrecognised outbound transfer of £2,450 (transaction reference [TRANSACTION_ID_1]). The transaction completed successfully and was linked to a login from IP address [IP_ADDRESS_1] at 02:17 AM, outside the [PERSON_1]’s normal activity window.

The [PERSON_1] retains access to the account and no additional suspicious transactions have been identified. The case is currently under review by the [ORG_1], and temporary preventative controls have been applied.

Please provide:

A short executive summary (3–5 sentences)

Any immediate financial or operational considerations

Current status and next steps


In [46]:
model_prompt = model_instruction + "\n" + masked_text
print(model_prompt)


Note: The following message contains intentionally masked information using placeholders 
(e.g. [PERSON_1], [ACCOUNT_ID_1], [TRANSACTION_ID_1]).

Please:
- Treat each placeholder as a consistent anonymous entity.
- Refer back to the same placeholder when discussing actions or events related to it.
- Do not invent names, identities, or additional personal details.
- Use the masked information as provided to respond to the user’s query.
Please draft a concise internal summary for a board-level report based on the information below.

Case details:

On [DATE_1], a [PERSON_1] reported an unrecognised outbound transfer of £2,450 (transaction reference [TRANSACTION_ID_1]). The transaction completed successfully and was linked to a login from IP address [IP_ADDRESS_1] at 02:17 AM, outside the [PERSON_1]’s normal activity window.

The [PERSON_1] retains access to the account and no additional suspicious transactions have been identified. The case is currently under review by the [ORG_1], and t

In [39]:
txt = """I’m reporting a transaction on my account that I don’t recognise.

My name is Sarah. On 14 January 2026, I noticed an outbound bank transfer of £2,450 from my account. The transaction reference shown in my app is FT-98437261. The payment appears to have completed successfully, but I do not remember authorising it.

When checking my recent activity, I also saw that my account was logged into from a new IP address (185.73.44.219) around 02:17 AM, which is outside my usual usage hours. I normally only access my account during the day, and I do not recognise this device or location.

I still have access to my account, and at the moment I haven’t noticed any other suspicious transactions. This is the first time something like this has happened to me.

Could you please investigate this transaction and advise what steps will be taken next? I would also like to know if I should take any immediate action to protect my account while the review is ongoing.


"""

masked_text, mapping, scores, spans = run_test(txt, threshold=0.7)
print(masked_text)

I’m reporting a transaction on my account that I don’t recognise.

My name is [PERSON_1]. On [DATE_1], I noticed an outbound bank transfer of £2,450 from my account. The transaction reference shown in my app is [TRANSACTION_ID_1]. The payment appears to have completed successfully, but I do not remember authorising it.

When checking my recent activity, I also saw that my account was logged into from a new IP address ([IP_ADDRESS_1]) around 02:17 AM, which is outside my usual usage hours. I normally only access my account during the day, and I do not recognise this device or location.

I still have access to my account, and at the moment I haven’t noticed any other suspicious transactions. This is the first time something like this has happened to me.

Could you please investigate this transaction and advise what steps will be taken next? I would also like to know if I should take any immediate action to protect my account while the review is ongoing.





In [3]:
_results = run_tests_from_json(path="tests.json", threshold=0.1, show_masked_text=False)
tests = load_tests_bundle(path="tests.json").get("tests", [])

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Loaded 14 tests from G:\Proxyon\Test\finalised\tests.json
Model: urchade/gliner_multi_pii-v1 | threshold=0.1
-
sample_1   | Recall: 100.0% | TypeAcc: 88.5% | Overall: 88.5% | FP: 0 (found 26, expected 26)
sample_2   | Recall: 96.2% | TypeAcc: 84.0% | Overall: 80.8% | FP: 0 (found 25, expected 26)
sample_3   | Recall: 92.3% | TypeAcc: 91.7% | Overall: 84.6% | FP: 1 (found 25, expected 26)
   FP by type: CARD_EXPIRY:1
sample_3   | Recall: 92.3% | TypeAcc: 91.7% | Overall: 84.6% | FP: 1 (found 25, expected 26)
   FP by type: CARD_EXPIRY:1
sample_4   | Recall: 83.3% | TypeAcc: 85.0% | Overall: 70.8% | FP: 0 (found 20, expected 24)
sample_5   | Recall: 80.0% | TypeAcc: 81.2% | Overall: 65.0% | FP: 1 (found 17, expected 20)
   FP by type: ORG:1
sample_6   | Recall: 80.0% | TypeAcc: 93.8% | Overall: 75.0% | FP: 0 (found 16, expected 20)
sample_7   | Recall: 78.9% | TypeAcc: 93.3% | Overall: 73.7% | FP: 0 (found 15, expected 19)
sample_8   | Recall: 83.3% | TypeAcc: 100.0% | Overall: 83.3% | F

In [4]:
test_num = 11
print(tests[test_num]["text"])#
print(20 * "---- Masked ----")
print(_results[test_num][2])

QUEUE: KYC
Agent: LS

Name on application: Yusuf Al-Mahdi

DOB: 1994/11/03

Home address:
Flat 7, Crescent House
114 Baker Street
London W1U 6TR

Contact yusuf.mahdi@securemail.uk
Phone 07990 443322

IP check 82.13.77.91
Session sess_4f91caa2

Bank info submitted:
Sort code 11-22-33
Account number 99887766
IBAN GB19 HLFX 1122 3399 8877 66

Refs
Transaction TRX-2026-01-16-310009
Customer CUST-REF-UK-220771
Ticket SUP-220771
Account ACC-4422911
Internal INT-091AF
---- Masked -------- Masked -------- Masked -------- Masked -------- Masked -------- Masked -------- Masked -------- Masked -------- Masked -------- Masked -------- Masked -------- Masked -------- Masked -------- Masked -------- Masked -------- Masked -------- Masked -------- Masked -------- Masked -------- Masked ----
QUEUE: KYC
Agent: LS

Name on application: [PERSON_1]

DOB: [DATE_OF_BIRTH_1]

Home address:
[UK_ADDRESS_1]

Contact [EMAIL_ADDRESS_1]
Phone [UK_PHONE_NUMBER_1]

IP check [IP_ADDRESS_1]
Session [SESSION_ID_1]

Ban